In [73]:
import time
import re
import shutil
import os
import sys

import threading

from skimage import io
import numpy as np
import matplotlib.pyplot as plt


In [6]:
#display patches
def show_image_grid(images: list, img_names: list,img_per_row: int = 2) -> None:
    n: int = len(images)
    n_cols = int(n/img_per_row)
    print_size = int(images[0].shape[0]/10)
    f = plt.figure(figsize=(print_size,print_size))
    print("#images[%d] n_rows[%d] n_cols[%d] print_size[%d]" % (n,img_per_row,n_cols,print_size))
    for i in range(n):
        #for j in range(img_per_row):
        #print(i,img_per_row,n_cols)
        # Debug, plot figure
        f.add_subplot( n_cols, img_per_row, i+1)
        plt.title(img_names[i])
        plt.axis('off')
        #plt.imshow(images[(i*img_per_row+j)])
        plt.imshow(images[i])
    plt.show(block=True)


In [55]:
def get_img_files_from_dir(ddir: str,file_name_pattern: str = None):
    #print("dir: {} pattern: {}".format(ddir,file_name_pattern))
    files = os.listdir(ddir)
    m_files = []
    i=0
    for filename in files:
        if file_name_pattern:
            m = re.findall(file_name_pattern,filename)
            #print(m)
            if m:
                m_files.append(m[0])
            else:
                continue
        else:
            m_files.append(filename)
        #print(filename)
        i += 1
        
    return m_files
print(get_img_files_from_dir('/disk1/data1/data/px_he1/img1/',file_name_pattern='^(.*?)\.jpg$'))

['IDRiD_50_2659']


In [45]:
def show_images_from_dir(odir: str, graph_flag: bool = False, ddir: str = '/disk1/data1/data/px_he1/img1/'):
    update_flag = False
    files = get_img_files_from_dir(odir,'^(.*?)\.jpg$)')
    m_files = []
    i=0
    for filename in files:
        #if i > 3:
        #    break
        #print(fnmatch.fnmatch(filename, file_pattern))
        #print(filename)
        '''
        '_groundtruth_(1)_img1_IDRiD_50_2659.jpg_e336bef0-f6c1-4342-8b5e-b45116c22743.jpg'
         img1_original_IDRiD_50_2659.jpg_031288ab-4d12-4a32-8c64-b36d0d0a2748.jpg

        '''
        o_img_fl = filename        
        if update_flag:
            if fnmatch.fnmatch(filename, o_file_pattern):
                m = re.findall('^.*?_(IDRiD_.*?)\.jpg_(.*?\.jpg)$',filename)            
                if m:
                    #print(m[0][0],m[0][1])
                    t_f_name = m[0][0] + '_' + m[0][1]
                    shutil.move(odir + o_img_fl,da_image_dir + t_f_name)
                #o_img = plt.imread( odir + o_img_fl)
            if fnmatch.fnmatch(filename, gt_file_pattern):
                m = re.findall('^.*?_(IDRiD_.*?)\.jpg_(.*?\.jpg)$',filename)            
                if m:
                    #print(m[0][0],m[0][1])
                    t_f_name = m[0][0] + '_' + m[0][1]
                    shutil.move(odir + o_img_fl,da_gt_dir + t_f_name)
        else:
            #print("o_file_pattern matched...shape is ",o_img.shape)
            o_img = io.imread( odir + o_img_fl, as_gray=True)
            print("#####",o_img_fl,o_img.size,o_img.shape)
            show_image_grid([o_img], [o_img_fl], img_per_row=1)

            images.append(o_img)
            img_names.append(o_img_fl)
            if graph_flag:
                if len(o_img.shape) > 2:
                    for j in range(o_img.shape[2]):
                        Z = o_img[:,:,j]
                        if o_img.shape[0] < o_img.shape[1]:
                          Z = Z.transpose((1,0))
                        plot_3d(Z,w=o_img.shape[0],h=o_img.shape[1],title=str(j))
                else:
                    plot_3d(Z=o_img,w=o_img.shape[0],h=o_img.shape[1],title='Greyscale=')
        
        if i % 1000 == 0:
            print(" %d images processed..." % (i))
        i += 1
        

In [42]:
def random_erase_np_v2(img, probability = 0.5, sl = 0.001, sh = 0.05, r1 = 0.3):
    height = img.shape[0]
    width = img.shape[1]
    channel = img.shape[2]
    area = width * height

    #erase_area_low_bound = np.round( np.sqrt(sl * area * r1) ).astype(np.int)
    #erase_area_up_bound = np.round( np.sqrt((sh * area) / r1) ).astype(np.int)
    erase_area_low_bound = np.round( np.sqrt(sl * area)).astype(np.int)
    erase_area_up_bound = np.round( np.sqrt(sh * area)).astype(np.int)
    if erase_area_up_bound < height:
        h_upper_bound = erase_area_up_bound
    else:
        h_upper_bound = height
    if erase_area_up_bound < width:
        w_upper_bound = erase_area_up_bound
    else:
        w_upper_bound = width

    h = np.random.randint(erase_area_low_bound, h_upper_bound)
    w = np.random.randint(erase_area_low_bound, w_upper_bound)

    x1 = np.random.randint(0, height+1 - h)
    y1 = np.random.randint(0, width+1 - w)

    x1 = np.random.randint(0, height - h)
    y1 = np.random.randint(0, width - w)
    img[x1:x1+h, y1:y1+w, :] = np.random.randint(0, 255, size=(h, w, channel)).astype(np.uint8)
    return img


In [71]:
def generate_random_erase_samples( mythread_name: str, f_list: list, ddir: str, odir: str, sample_size: int, file_ext: str = '.jpg'):
    i = 0
    start = time.time()
    for s_ind in range(sample_size):       
        for file in f_list:
            #print(ddir + file + file_ext)
            img = io.imread( ddir + file + file_ext)
            t_img = random_erase_np_v2(img,)
            io.imsave( odir + file + '_' + str(s_ind) + file_ext,t_img)
            i += 1
            #show_image_grid([t_img], [file], img_per_row=1)
            #print("file: {} time taken: {} img.shape: {} t_img.shape: {}".format(file,time.time()-start,img.shape,t_img.shape))
    print("Thread: {} generated {} images in {}...".format(mythread_name,i,time.time()-start))

In [77]:
def get_random_erase_samples( ddir: str, odir: str = '/tmp/images/', sample_size: int = 1, file_ext: str = '.jpg'):    
    if not os.path.exists(odir):
        os.mkdir(odir)
    files = get_img_files_from_dir( ddir, file_name_pattern = "^(.*?)\\" + file_ext + "$")  
    #print(files)
    files = np.array(files,dtype='str')
    if files.shape[0] > 1 and files.shape[0] % 2 != 0:
        files = files[:-1] #skip last element to make it proper even number of array content
    #print(files,files.shape[0])
    files = np.split(files,2)
    i = 0
    for f_list in files:
        #print(f_list)
        #generate_random_erase_samples(f_list=f_list,ddir=ddir,odir=odir,sample_size=sample_size)
        try:
            x = threading.Thread( target=generate_random_erase_samples, args=("Thread-%d" % (i), \
                                                                    f_list, \
                                                                    ddir, \
                                                                    odir, \
                                                                    sample_size, ) )
            
            x.start()
            #x.join()
            i += 1
        except:
            print("Error: unable to start thread")

get_random_erase_samples(ddir = '/disk1/data1/data/px_he1/img1/',sample_size=10)

['IDRiD_50_2659', 'IDRiD_50_2342', 'IDRiD_50_2501', 'IDRiD_50_1744', 'IDRiD_72_3647']
['IDRiD_50_2659' 'IDRiD_50_2342' 'IDRiD_50_2501' 'IDRiD_50_1744'] 4
Thread: Thread-1 generated 20 images in 0.09718608856201172...
Thread: Thread-0 generated 20 images in 0.10088205337524414...
